In [1]:
# -*- coding: utf-8 -*-
import os 
import glob
import pickle
from collections import defaultdict
import itertools
from pathlib import Path
import pandas as pd
import numpy as np
import re
from representations.utils import load_corpus
from datetime import datetime
from text_preprocessing import *
import spacy
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s :: %(levelname)s :: %(message)s')

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Pre-processing functions for the protocols

In [235]:
class Protocols():
    data_folder = Path('C:\\Users\\Tobias\\Documents\\Uni Mannheim\\Master Thesis\\notebooks\\data')
    
    def __init__(self, filename):
        filename = self.data_folder / filename
        file = open(filename, 'r', encoding='utf-8')
        self.text = file.readlines()
        
    def __iter__(self):
        return iter(self.text)
    
    def __str__(self):
        return " ".join(self.text)
    
    # Remove line breaks
    def remove_linebreaks(self, lines):
        self.text = [re.sub(r'\n', '', line).strip() for line in lines]
        return self

    # Remove punctuation: Leave out dash sign so that the composite words in German language stay as is
    def remove_punctuation(self, lines):    
        pattern = re.compile('[%s]' % re.escape('!"#$&\'()*+,./:;<=>?@®©[\\]^_`{|}~„“«»'))
        # pattern = re.compile('[%s]' % re.escape(string.punctuation))
        self.text = [re.sub(pattern,'', line).strip() for line in lines]
        return self 

    # Remove the digit 1 appending many nouns (e.g. Schlachtvieh1, Finanzwesen1) or is in between two words that don't belong
    # together (e.g. nimmt1die)

    def remove_noisy_digits(self, lines):
        temp = [re.sub(r'\b1(?P<quote>[A-Za-z]+)\b', '\g<quote>', line) for line in lines]
        temp = [re.sub(r'\b(?P<quote>[A-Za-z]+)1\b', '\g<quote>', line) for line in temp]
        self.text = [re.sub(r'\b(?P<quote>[A-Za-zßäöü]+)1(?P<name>[A-Za-zßäöü]*)\b', '\g<quote> \g<name>', line) for line in temp]
        return self 

    # Substitute every digit by a placeholder in order to harmonize corpus --> Need to insert space after each placeholder
    # as there are a lot of digits attached with words
    def replace_digits(self, lines):
        self.text = [re.sub(r'\d+',' 0 ' ,line).strip() for line in lines]
        return self 
    
    # After replacing digits (both arabic and roman) there will be many sequences of consecutive placeholders
    # (e.g. \\rom \\za \\rom \\rom \\za) for example when a passage in the law is cited --> reduce to a single placeholder
    
    def replace_roman_numerals(self,lines):
        # Need to run 4 different regex operations in order to capture all roman numerals
        
        # for roman numerals that appear in the middle of a line
        temp = [re.sub(r'\b\sM{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\s\b',' \rom ', line, flags=re.IGNORECASE) 
                for line in lines]
        # for roman numerals that are single element of a line
        temp = [re.sub(r'\b^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$\b', '\rom', line, flags=re.IGNORECASE)
                for line in temp]
        # for roman numerals that appear at the beginning of a line but line contains other elements 
        temp = [re.sub(r'^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\s' , '\rom ', line, flags=re.IGNORECASE)
                for line in temp]
        # for roman numerals that appear at the end of a line but line contains other elements 
        self.text = [re.sub(r'\sM{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$', ' \rom', line, flags=re.IGNORECASE)
                for line in temp]
        return self 
    
    def remove_dash_and_minus_signs(self, lines):
        # lone standing dash signs at beginning, middle and end of line
        temp = [[re.sub(r'\s(-|—|–|\s)+\s', ' ', line) for line in doc] for doc in lines]
        temp = [[re.sub(r'^(-|—|–|\s)+\s', '', line) for line in doc] for doc in temp]
        self.text = [[re.sub(r'\b\s(-|—|–|\s)+$', '', line) for line in doc] for doc in temp]
        return self
    
    def reduce_numerical_sequences(self,lines): 
        self.text = [re.sub(r'((0)\s?){2,}', '\\2 ', line).strip() for line in lines]
        return self 
    
    def remove_single_letters(self, lines):
        self.text = [re.sub(r'\b[A-Za-zÄÖÜäöüß]\b','', line) for line in lines]
        return self 
        
    # Remove double spacing
    def remove_double_spaces(self, lines):
        self.text = [re.sub('\s\s+',' ', line).strip() for line in lines]
        # protocols = re.sub('\s\s+',' ', protocols)
        return self 
    def removeUmlauts(self,lines):
        temp = [line.replace('ä', 'ae', re.IGNORECASE) for line in lines]
        temp = [line.replace('ö', 'oe', re.IGNORECASE) for line in temp]
        temp = [line.replace('ü', 'ue', re.IGNORECASE) for line in temp]
        self.text = [line.replace('ß', 'ss', re.IGNORECASE) for line in temp]
        return self

In [3]:
# dirname = 'data/WP_15'
# if not os.path.exists(f'{dirname}_processed'):
#     os.makedirs(f'{dirname}_processed')
# files_total = len(os.listdir(dirname))
# logging.info(f'{files_total} files were found.')
# border = round(files_total / 10, 2)
# for num in range(1,files_total+1):
#     try:
#         text = open(os.path.join(dirname, f'{num}_sents.txt'), encoding='utf-8').readlines()
#         protocols_text = extract_protocol(text)
#         text = remove_linebreaks(protocols_text)
#         punt_removed_text = remove_punctuation(text)
#         text = remove_double_spaces(punt_removed_text)
#         noisy_digits_removed_text = remove_noisy_digits(text)
#         digits_replaced = replace_digits(noisy_digits_removed_text)
#         text = reduce_numerical_sequences(text)
#         text = remove_dash_and_minus_signs(text)
#         text = remove_double_spaces(text)
#         text = remove_empty_lines(text)
#         text = [removeGermanChainWords(line) for line in text]
#         text = [expandCompoundToken(line) for line in text]
#         text = [lemmatizer.lemmatize(line) for line in text]
#         text = [spell_checker.correct(line) for line in text]
#         text = [[tok.lower() for tok in line] for line in text]
#     #             text = itertools.chain.from_iterable(text)
#         save_as_line_sentence(text, f'{self.dirname}_processed/{num}.txt')
#         i += 1
#         if i % border == 0:
#           logging.info('Processing {:03.1f} percent finished'.format(float((i/(files_total)) * 100)))

#     except FileNotFoundError:
#         print(f'File {num} was not found.')

In [110]:
class CreateProceedingsCorpus:
    data_folder = Path('data')
    def __init__(self, dirname):
        self.dirname = str(self.data_folder / dirname)
    def __iter__(self):
        num_files = len(glob.glob(f'{self.dirname}/*.txt'))
        for num in range(1,num_files+1):
            for file in glob.glob(f'{self.dirname}/{num}_sents.txt'):
                text = open(file, encoding='utf-8').readlines()
                yield text

In [111]:
class PlenarProceedings():
    data_folder = Path('C:\\Users\\Tobias\\documents\\Uni Mannheim\\Master Thesis\\notebooks\\data')
    
    def __init__(self, dirname):
        self.proceedings = list(CreateProceedingsCorpus(self.data_folder / dirname))
        
    def __iter__(self):
        return iter(self.proceedings)
    
    def __str__(self):
        return " ".join(self.proceedings)
            
 # Remove line breaks
    def remove_linebreaks(self):
        self.proceedings = [[re.sub(r'[\n\t]', '', line).strip() for line in doc] for doc in self.proceedings]
        return self
    
    def remove_punctuation(self):    
        pattern = re.compile('[%s]' % re.escape('!"#$&\'()*+,./:;<=>?@®©[\\]^_`{|}~„«»'))
        self.proceedings = [[re.sub(pattern,'', line).strip() for line in doc] for doc in self.proceedings]
        return self 
    
    def remove_double_spaces(self):
        self.proceedings = [[re.sub('\s\s+',' ', line).strip() for line in doc] for doc in self.proceedings]
        return self
    
#     def replace_measures(self):
#         temp = [[re.sub(r'%', 'Prozent') for line in doc] for doc in self.proceedings]
#         temp = [[re.sub(r'(°o|‰|%o)', 'Promille') for line in doc] for doc in self.proceedings]
    
    def remove_noisy_digits(self):
        temp = [[re.sub(r'\b1(?P<quote>[A-Za-z]+)\b', '\g<quote>', line) for line in doc] for doc in self.proceedings]
        temp = [[re.sub(r'\b(?P<quote>[A-Za-z]+)1\b', '\g<quote>', line) for line in doc] for doc in temp]
        temp = [[re.sub(r'\b(?P<quote>[A-Za-zßäöü]+)1(?P<name>[A-Za-zßäöü]*)\b', '\g<quote> \g<name>', line) for line in doc] for doc in temp]
        self.proceedings = temp
        return self
    
    def replace_digits(self):
        self.proceedings = [[re.sub(r'\d+',' 0 ' ,line).strip() for line in doc] for doc in self.proceedings]
        return self 
    
    def replace_roman_numerals(self):
        # Need to run 4 different regex operations in order to capture all roman numerals
        
        # for roman numerals that appear in the middle of a line
        temp = [[re.sub(r'\b\sM{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\s\b',' 10 ', line, flags=re.IGNORECASE) 
                for line in doc] for doc in self.proceedings]
        # for roman numerals that are single element of a line
        temp = [[re.sub(r'\b^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$\b', '10', line, flags=re.IGNORECASE)
                for line in doc] for doc in temp]
        # for roman numerals that appear at the beginning of a line but line contains other elements 
        temp = [[re.sub(r'^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\s' , '10 ', line, flags=re.IGNORECASE)
                for line in doc] for doc in temp]
        # for roman numerals that appear at the end of a line but line contains other elements 
        self.proceedings = [[re.sub(r'\sM{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$', ' 10', line, flags=re.IGNORECASE)
                for line in doc] for doc in temp]
        return self 
    
    def reduce_numerical_sequences(self): 
        self.proceedings = [[re.sub(r'((0)\s?){2,}', '\\2 ', line).strip() for line in doc] for doc in self.proceedings]
        return self
    
    def remove_dash_and_minus_signs(self):
        # lone standing dash signs at beginning, middle and end of line
        temp = [[re.sub(r'\s(-|—|–|\s)+\s', ' ', line) for line in doc] for doc in self.proceedings]
        temp = [[re.sub(r'^(-|—|–|\s)+\s', '', line) for line in doc] for doc in temp]
        self.proceedings = [[re.sub(r'\b\s(-|—|–|\s)+$', '', line) for line in doc] for doc in temp]
        return self
    
    def remove_single_letters(self):
        self.proceedings = [[re.sub(r'\b[A-Za-zÄÖÜäöüß]\b','', line) for line in doc] for doc in self.proceedings]
    
    def remove_empty_lines(self):
        self.proceedings = [[line for line in doc] for doc in self.proceedings if len(line) >0]
        return self 

In [153]:
period_1 = PlenarProceedings('WP_1')

In [154]:
period_1.remove_punctuation()\
        .remove_double_spaces()
period_1.proceedings = [extract_protocol(doc) for doc in period_1.proceedings]

In [155]:
period_1.remove_noisy_digits()\
        .replace_digits()\
        .reduce_numerical_sequences()\
        .remove_dash_and_minus_signs()\
        .remove_double_spaces()

In [241]:
protocols_1 = Protocols('1895.corr.seg')
protocols_2 = Protocols('1918.corr.seg')
protocols_3 = Protocols('1933.corr.seg')
protocols_4 = Protocols('1942.corr.seg')

In [237]:
all_protocols = (protocols_1, protocols_2, protocols_3, protocols_4)

In [ ]:
protocols_1.remove_punctuation(protocols_1.text)\
            .remove_double_spaces(protocols_1.text)\
            .remove_noisy_digits(protocols_1.text)\
            .remove_dash_and_minus_signs(protocols_1.text)

In [239]:
all_protocols[0].text[:200]

['Stenographische Berichte über die Verhandlungen des Reichstags',
 'VII',
 'Legislaturperiode',
 'V Session 188SSV',
 'Erster Band',
 'Von der Eröffnungssitzung am 22',
 'Oktober bis zur 27',
 'Sitzung am 2',
 'Dezember 1889',
 'Von Seite 1 bis 631',
 'Voran Sprechregister Seite IX ff',
 'Das Sachregister befindet sich am Schluß des zweiten das Mitgliederverzeichniß — als Nr',
 '1 der Anlagen — am Anfang des dritten Bandes Anlagebandes',
 'Die Vorlagen der verbündeten Regierungen die Kormnisstonsberichte c sind in dem Anlageband der Stenographischen Berichte dritter Band abgedruckt welcher im Verlage von Julius Sittenfeld in Berlin Mauerstraße Nr',
 '6365 erscheint',
 'Das Post-Abonnement auf diese Anlagen ist besonders zu bestellen',
 'Berlin 1890',
 'Druck und Verlag der Norddeutschen Buchdruckerei und Verlags-Anstalt',
 'Berlin 8',
 'Wilhelmstraße Nr',
 '32',
 'X Inhaltsverzeichnilj',
 'Allerhöchste Verordnung vom 30',
 'September 1889 die Einberufung des Reichstags betreffend 1',


In [ ]:
# Approach new
 text = open(os.path.join(tpath, self.input),'r', encoding='utf-8').readlines()
 text = remove_punctuation(text)
 text = remove_double_spaces(text)
 text = remove_noisy_digits(text)
#  text = replace_digits(text)
#  text = remove_double_spaces(text)
#  text = reduce_numerical_sequences(text)
 text = remove_dash_and_minus_signs(text)
#  text = filter_lines(text)
#  text = [removeGermanChainWords(line) for line in text]
 logging.info('Chainword splitting finished')
#  text = [remove_hyphens_pre_and_appending(line) for line in text]
 text = [lemmatizer.lemmatize(line) for line in text]
 logging.info('Lemmatizing finished')
 text = [lowercase(line) for line in text]
 text = [removeUmlauts(line) for line in text]
 text = [harmonizeSpelling(line) for line in text_preprocessing]
 save_as_line_sentence(text, f'{self.input[:-4]}_processed')
 logging.info('Processing finished')

#### Find end patterns

In [13]:
# Check if "Meine Damen und Herren" and "Schluss der Sitzung" are good markers for session ends and endings
ends = 0
i = 0
for p in period_18.proceedings:
    sitzung_end = False
    for line in p:
        m = re.search(end_patterns, line, re.IGNORECASE)
        if m:
            ends += 1
#             print(m)
            sitzung_end = True
            break
    if not sitzung_end:
        print(i)
    i += 1

In [14]:
ends

245

#### Find start patterns

In [26]:
# Check if "Meine Damen und Herren" and "Schluss der Sitzung" are good markers for session starts and endings
starts = 0
i = 0
for p in period_18.proceedings:
    sitzung_start = False
    for line in p:
        m = re.search(start_patterns, line, re.IGNORECASE)
        if m:
            starts += 1
            sitzung_start = True
            break
    if not sitzung_start:
        print(i)
    i += 1

134
212
215
222


In [18]:
starts
ends

241

245

In [22]:
starts = 0
ends = 0
i = 0
prots_von_anfang = [extract_protocol_new_new(doc) for doc in period_18
                    .proceedings]

134: no start
212: no start
215: no start
222: no start


##### Removing abbreviations 

Stand der Protokolle: German chainwords removed ja, abbreviations removed nein, spelling correction nein

In [317]:
prots_new = [extract_protocol_new(doc) for doc in period_14.proceedings]

### Reichstag Proceedings

In [ ]:
def save_protocols(p,number):
    with open('obj/protocols_{}.pkl'.format(number), 'wb') as f:
            pickle.dump(p.text, f, pickle.HIGHEST_PROTOCOL)

In [44]:
save_protocols(protocols_1,1)
save_protocols(protocols_2,2)
save_protocols(protocols_3,3)
save_protocols(protocols_4,4)

## Extract the minutes/protocols from the Reichstag sessions

In [3]:
protocols_1 = load_corpus('protocols_1.pkl')
protocols_2 = load_corpus('protocols_2.pkl')
protocols_3 = load_corpus('protocols_3.pkl')
protocols_4 = load_corpus('protocols_4.pkl')

### Some experimentation on extraction methods

In [13]:
# Chop whole corpus into documents by searching for keywords 'Die Sitzung ist eröffnet' and 'Schluss der Sitzung' ob
# Save one document per meeting
def extract_meeting_protocols_linebyline(docs):
    
    if not os.path.exists('protocols_1_test2'):
        os.makedirs('./protocols_1_test2')
    matches = list(re.finditer(r'\d+ Sitzung (?:am )?(?:Montag |Dienstag |Mittwoch |Donnerstag |Freitag |Sonnabend |Sonntag )(?:den )(\d+ (?:Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|November|Dezember) \d{4})', 
          docs, flags=re.IGNORECASE))
    
#     start_index = matches[0].start()
#     sitzung_end = re.search(r'(?:Schluß|Sckluß) der Sitzung (?:um\s)?\d+ Uhr', docs[start_index:],re.IGNORECASE)
    
    i=0
    for j in range(0,len(matches)-1):                               
        if (j>0 and matches[j][0] == matches[j-1][0]):
            continue
        else:
            start_index = matches[j].start()
            print(start_index)
            sitzung_end = re.search(r'(?:Schluß|Sckluß) der Sitzung (?:um\s)?\d+ Uhr', docs[start_index:],re.IGNORECASE)
            print(sitzung_end)
            end_index = sitzung_end.end()
            print(end_index)
            temp_doc = open('./protocols_1_test2/doc_{}.txt'.format(i), 'w',  encoding='utf-8')
            temp_doc.write(docs[start_index:end_index])
#             start_index = matches[j+1].start()
            i+=1
        
    temp_doc.close()
    print('Screening done!')

In [106]:
def extract_session_recursive(docs, number):
    match = re.search(r'\d+ Sitzung (am )?(?:Montag|Dienstag|Mittwoch|Donnerstag|Freitag|Sonnabend|Sonntag) den \d+ (?:Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|November|Dezember) \d{4}',
                    docs,flags=re.IGNORECASE)
    matches = re.findall(r'\d+ Sitzung (am )?(?:Montag|Dienstag|Mittwoch|Donnerstag|Freitag|Sonnabend|Sonntag) den \d+ (?:Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|November|Dezember) \d{4}',
                    docs,flags=re.IGNORECASE)
    
    for m in matches:
    if match:
        start_index = match.end()
        sitzung_end = re.search(r'(?:Schluß|Sckluß) der Sitzung (?:um\s)?\d+ Uhr', docs[start_index:],re.IGNORECASE)
        if sitzung_end:
            end_index = sitzung_end.end()
            temp_doc = open('./protocols_1_test_rec/doc_{}.txt'.format(number), 'w',  encoding='utf-8')
            temp_doc.write(docs[start_index:end_index])
            temp_doc.close()
            extract_session_recursive(docs[end_index:], number+1)
        else:
            print('Not able to find the end of the session.')
    else:    
        print('No more sessions found!')

In [88]:
with open('C:/Users/Tobias/Documents/Uni Mannheim/Master Thesis/notebooks/protocols_adapted_3_3/doc_22_2.txt') as f:
    f.close()

In [ ]:
lines = 'what up bitches do not even think you deserve of me'.split()

In [221]:
# Chop whole corpus into documents by searching for keywords 'Die Sitzung ist eröffnet' and 'Schluss der Sitzung' ob
# Save one document per meeting
doc_num = 0
def extract_meeting_protocols_recursive(lines,number):
#     sitzung = False
    global doc_num
    if not os.path.exists('./protocols_recursive_{}'.format(number)):
        os.makedirs('./protocols_recursive_{}'.format(number))
    temp_doc = None
    
#     if number <= 3:                 

#         start_pattern = re.compile(r'Die Sitzung ist eröffnet',re.IGNORECASE)
#     else:
#         start_pattern = re.compile(r'Die Sitzung wird um \d+ Uhr(?:\s\d+ Minute)?n?(?:\sabends)? durch den Präsidenten eröffnet',
#                                   re.IGNORECASE)

    for i in range(len(lines)):
        sitzung_restart = restart_pattern.search(lines[i])
        sitzung_start = start_pattern.search(lines[i])
        
        if sitzung_restart:
#             sitzung = True
            temp_doc = open('./protocols_recursive_{}/doc_{}.txt'.format(number,doc_num), 'a',  encoding='utf-8')
            temp_doc.write(lines[i])
            temp_doc.write('\n')
            for j in range(len(lines[i:])):
                temp_doc.write(lines[i:][j])
                temp_doc.write('\n')
                sitzung_end = end_pattern.search(lines[i:][j])
                sitzung_abgebrochen = re.search(r'Die Sitzung wird um \d+ Uhr (?:\d+ Minute)?n? abgebrochen', lines[i:][j],
                                   re.IGNORECASE) 
                if sitzung_end or sitzung_abgebrochen:
    #             temp_doc.write(sitzung_start[0] + '\n')
#                 sitzung = False
                    border = i + j +1
                    extract_meeting_protocols_recursive(lines[border:],number)
        
        if sitzung_start:
            doc_num += 1
#             sitzung = True
            print(doc_num)
            temp_doc = open('./protocols_recursive_{}/doc_{}_2.txt'.format(number,doc_num), 'w',  encoding='utf-8')
            for j in range(len(lines[i:])):
                temp_doc.write(lines[i:][j])
                temp_doc.write('\n')
                sitzung_end = end_pattern.search(lines[i:][j])
                sitzung_abgebrochen = re.search(r'Die Sitzung wird um \d+ Uhr (?:\d+ Minute)?n? abgebrochen', lines[i:][j],
                                   re.IGNORECASE) 
                if sitzung_end or sitzung_abgebrochen:
    #             temp_doc.write(sitzung_start[0] + '\n')
#                 sitzung = False
#                     temp_doc.close()
                    border = i + j +1
                    extract_meeting_protocols_recursive(lines[border:],number)
    
#     temp_doc.close()
        

In [222]:
extract_meeting_protocols_recursive(protocols_4,4)

KeyboardInterrupt: 

## The actual processing of the lines

In [4]:
start_patterns_reichstag = '|'.join(
                ['(Z|I)ch eröffne die (\d+ |erste )?(\n )?Sitzung',
                 'Di(e|s) (\d+ |erste )?(\n )?Sitzung (\n )?ist (\n )?erö(f|s)(f|s)net',
                 'Die Sitzung-ist eröffnet',
                 'eröffne ich (hiermit )?die Sitzung',
                 'Ich eröffne die \d+$',
                 '(Z|I)ch erkläre die (\d+ |erste )?Sitzung für eröffnet']
                        )

restart_patterns_reichstag = '|'.join(
                ['Die (\n )?Sitzung (\n )?ist wieder (\n )?eröffnet',
                 'Ich eröffne die Sitzung wieder',
                 'Ich eröffne die Sitzung von neuem',
                 'Ich eröffne die Sitzung noch einmal'
                ])
                        
end_patterns_reichstag = '|'.join(
                ['(Schluß|Schluss|Sckluß) der Sitzung (um )?\d+ Uhr',
                 'Die Sitzung ist geschlossen',
                 'Die Sitzung ist geschloffen',
                 'Ich schließe die (\n )?Sitzung'
                    ])

In [5]:
# Use whatever comes first as start of the session protocol
start_pattern_reichstag = re.compile(f"({start_patterns_reichstag})", re.IGNORECASE)
restart_pattern_reichstag = re.compile(f"({restart_patterns_reichstag})", re.IGNORECASE)
end_pattern_reichstag = re.compile(f"({end_patterns_reichstag})", re.IGNORECASE)

In [6]:
# Chop whole corpus into documents by searching for keywords 'Die Sitzung ist eröffnet' and 'Schluss der Sitzung' ob
# Save one document per meeting
def extract_meeting_protocols_reichstag(lines,number):
    i = 0
    sitzung = False
    
    if not os.path.exists('./protocols_{}'.format(number)):
        os.makedirs('./protocols_{}'.format(number))
    temp_doc = None
    
    if number > 3:                 
        start_pattern_reichstag = re.compile(r'Die Sitzung wird um \d+ Uhr(?:\s\d+ Minute)?n?(?:\sabends)? durch den Präsidenten eröffnet',
                                  re.IGNORECASE)
    else:
        start_pattern_reichstag = re.compile(f"({start_patterns_reichstag})", re.IGNORECASE)
        
    restart_pattern_reichstag = re.compile(f"({restart_patterns_reichstag})", re.IGNORECASE)
    end_pattern_reichstag = re.compile(f"({end_patterns_reichstag})", re.IGNORECASE)
    for line in lines:
        sitzung_restart = restart_pattern_reichstag.search(line)
        sitzung_start = start_pattern_reichstag.search(line)
        sitzung_end = end_pattern_reichstag.search(line)
        sitzung_abgebrochen = re.search(r'Die Sitzung wird um \d+ Uhr (?:\d+ Minute)?n? abgebrochen', line,
                               re.IGNORECASE)
        
        if sitzung_restart:
            sitzung = True
            temp_doc.write(line)
            temp_doc.write('\n')
            continue
        if sitzung_start:
            i += 1
            if i % 100 == 0:
                logging.info(f'{i} documents extracted')
            
            sitzung = True
            temp_doc = open('./protocols_{}/doc_{}.txt'.format(number,i), 'w',  encoding='utf-8')
            temp_doc.write(line)
            temp_doc.write('\n')
            continue
        if sitzung_end or sitzung_abgebrochen:
            if temp_doc is not None:
                temp_doc.write(line)
                temp_doc.write('\n')
                sitzung = False
                continue

        if sitzung:
            if len(line) > 1:
                temp_doc.write(line)   
                temp_doc.write('\n')
    
    logging.info('Screening done!')
    logging.info(f'{i} documents extracted in total.')
    temp_doc.close()

Special regex for the protocols in 1942.corr.seg:

- Die Sitzung wird um 0 Uhr(?:\s0 Minuten){0,1}(?:\sabends){0,1} durch den Präsidenten eröffnet'

Weitere Alternativen, um noch mehr Protokolle zu finden:

1895:

- Ich erkläre die Sitzung für eröffnet
- Der Alterspräsident XY eröffnet die Sitzung um X Uhr Y Minuten
- Der Präsident XY eröffnet die Sitzung um X Uhr Y Minuten (Aktiv)
- Die Sitzung wird um X Uhr Y Minuten durch den Präsidenten XY eröffnet.

Der Aufwand ist extrem hoch für die größeren Corpora, deshalb erstmal hinten anstellen und bei alter Exraxction method für corpora 1-3 bleiben

In [55]:
print('Anzahl eröffnet: {}'.format(len(eröffnete_sitzungen_1)))
print('Anzahl geschlossen: {}'.format(len(geschlossene_sitzungen_1)))

Anzahl eröffnet: 77
Anzahl geschlossen: 3269


In [56]:
print('Anzahl eröffnet: {}'.format(len(eröffnete_sitzungen_2)))
print('Anzahl geschlossen: {}'.format(len(geschlossene_sitzungen_2)))

Anzahl eröffnet: 45
Anzahl geschlossen: 4635


In [57]:
print('Anzahl eröffnet: {}'.format(len(eröffnete_sitzungen_3)))
print('Anzahl geschlossen: {}'.format(len(geschlossene_sitzungen_3)))

Anzahl eröffnet: 93
Anzahl geschlossen: 2794


In [58]:
print('Anzahl eröffnet: {}'.format(len(eröffnete_sitzungen_4)))
print('Anzahl geschlossen: {}'.format(len(geschlossene_sitzungen_4)))

Anzahl eröffnet: 5
Anzahl geschlossen: 32


In [154]:
re.search(r'Die Sitzung wird um \d+ Uhr( \d+ Minute)?n?( abends)? durch den Präsi',
          'Die Sitzung wird um 3 Uhr 17 Minuten durch den Präsi')

<_sre.SRE_Match object; span=(0, 52), match='Die Sitzung wird um 3 Uhr 17 Minuten durch den Pr>

In [143]:
len('Die Sitzung wird um \d+ Uhr( \d+ Minute)?n?( abends)? durch den Praesidenten eröffnet')

85

In [176]:
#abgebrochene Sitzungen
abgebrochen = [r'i' for line in protocols_3 if re.findall(r'Die Sitzung wird um \d+ Uhr \d+ Minuten abgebrochen', line)]
print(len(abgebrochen))

1


In [ ]:
re.search('Die Sitzung wird um \d+ Uhr( d+ Minute)?n?( abends)? durch den Präsidenten ([\wäöü\WÄÖÜ\s]*)eröffnet',
          'Die Sitzung wird um 3 Uhr 17 Minuten durch den Präsidenten Löbe eröffnet Präsident')

In [156]:
anzahl_sitzungen = {}
start_patterns = ['(Z|I)ch eröffne die (\d+ |erste )?(\n )?Sitzung',
#                     'Di(e|s) (\d+ |erste )?(\n )?Sitzung (\n )?ist (\n )?erö(f|s)(f|s)net',
#                     'Die Sitzung-ist eröffnet',
#                     'eröffne ich die (erste )?Sitzung',
#                     'Ich erkläre die (\d+ |erste )?Sitzung für eröffnet',
                    'Ich eröffne die \d+$',
                    'Die Sitzung wird um \d+ Uhr( \d+ Minute)?n?( abends)? durch den Präsidenten eröffnet'
                    ]
# end_patterns = ['(Schluß|Schluss|Sckluß)( der Sitzung)? (um )?\d+ Uhr',
#                 '(Schluß|Schluss|Sckluß) der Sitzung (um )?\d+ Uhr',
#                 '(Schluß|Schluss|Sckluß)( der Sitzung)? \d+ Uhr'
#                ]
                            
for pattern in start_patterns:
    anzahl_sitzungen[pattern] = {}
    # Use whatever comes first as start of the session protocol
    start_pattern = re.compile(f"({pattern})", re.IGNORECASE)

    # How many times were both expressions found?
    eröffnete_sitzungen_1 = ['i' for line in protocols_1 if re.findall(start_pattern, line)]
    anzahl_sitzungen[pattern][1] = len(eröffnete_sitzungen_1)
    # geschlossene_sitzungen_1 = [r'i' for line in protocols_1 if re.findall(end_patterns, line)]

    # How many times were both expressions found?
    eröffnete_sitzungen_2 = ['i' for line in protocols_2 if re.findall(start_pattern, line)]
    anzahl_sitzungen[pattern][2] = len(eröffnete_sitzungen_2)
    # geschlossene_sitzungen_2 = [r'i' for line in protocols_2 if re.findall(end_pattern, line)]

    eröffnete_sitzungen_3 = ['i' for line in protocols_3 if re.findall(start_pattern, line)]
    # geschlossene_sitzungen_3 = [r'i' for line in protocols_3 if re.findall(end_pattern, line)]
    anzahl_sitzungen[pattern][3] = len(eröffnete_sitzungen_3)

    eröffnete_sitzungen_4 = ['i' for line in protocols_4 if re.findall(start_pattern, line)]
    # geschlossene_sitzungen_4 = [r'i' for line in protocols_4 if re.findall(end_pattern, line)]
    anzahl_sitzungen[pattern][4] = len(eröffnete_sitzungen_4)

# for pattern in end_patterns:
#     anzahl_sitzungen[pattern] = {}
#     # Use whatever comes first as start of the session protocol
#     end_pattern = re.compile(f"({pattern})", re.IGNORECASE)

#     # How many times were both expressions found?
#     eröffnete_sitzungen_1 = ['i' for line in protocols_1 if re.findall(end_pattern, line)]
#     anzahl_sitzungen[pattern][1] = len(eröffnete_sitzungen_1)
#     # geschlossene_sitzungen_1 = [r'i' for line in protocols_1 if re.findall(end_patterns, line)]

#     # How many times were both expressions found?
#     eröffnete_sitzungen_2 = ['i' for line in protocols_2 if re.findall(end_pattern, line)]
#     anzahl_sitzungen[pattern][2] = len(eröffnete_sitzungen_2)
#     # geschlossene_sitzungen_2 = [r'i' for line in protocols_2 if re.findall(end_pattern, line)]

#     eröffnete_sitzungen_3 = ['i' for line in protocols_3 if re.findall(end_pattern, line)]
#     # geschlossene_sitzungen_3 = [r'i' for line in protocols_3 if re.findall(end_pattern, line)]
#     anzahl_sitzungen[pattern][3] = len(eröffnete_sitzungen_3)

#     eröffnete_sitzungen_4 = ['i' for line in protocols_4 if re.findall(end_pattern, line)]
#     # geschlossene_sitzungen_4 = [r'i' for line in protocols_4 if re.findall(end_pattern, line)]
#     anzahl_sitzungen[pattern][4] = len(eröffnete_sitzungen_4)

In [175]:
anzahl_sitzungen

{'(Z|I)ch eröffne die (\\d+ |erste )?(\n )?Sitzung': {1: 103,
  2: 46,
  3: 43,
  4: 1},
 'Ich eröffne die \\d+$': {1: 1, 2: 12, 3: 1, 4: 0}}

In [ ]:
for i in range(len(all_protocols)):
    extract_meeting_protocols_reichstag(all_protocols[i],i+1)

In [7]:
extract_meeting_protocols_reichstag(protocols_1,1)
extract_meeting_protocols_reichstag(protocols_2,2)
extract_meeting_protocols_reichstag(protocols_3,3)
extract_meeting_protocols_reichstag(protocols_4,4)

2020-05-29 15:16:16,346 :: INFO :: 100 documents extracted
2020-05-29 15:16:33,169 :: INFO :: 200 documents extracted
2020-05-29 15:16:55,460 :: INFO :: 300 documents extracted
2020-05-29 15:17:14,418 :: INFO :: 400 documents extracted
2020-05-29 15:17:30,311 :: INFO :: 500 documents extracted
2020-05-29 15:17:42,834 :: INFO :: 600 documents extracted
2020-05-29 15:18:04,972 :: INFO :: 700 documents extracted
2020-05-29 15:18:19,066 :: INFO :: 800 documents extracted
2020-05-29 15:18:37,799 :: INFO :: 900 documents extracted
2020-05-29 15:18:53,810 :: INFO :: 1000 documents extracted
2020-05-29 15:19:10,377 :: INFO :: 1100 documents extracted
2020-05-29 15:19:31,165 :: INFO :: 1200 documents extracted
2020-05-29 15:19:51,058 :: INFO :: 1300 documents extracted
2020-05-29 15:20:18,389 :: INFO :: 1400 documents extracted
2020-05-29 15:20:34,301 :: INFO :: 1500 documents extracted
2020-05-29 15:20:50,863 :: INFO :: 1600 documents extracted
2020-05-29 15:21:07,977 :: INFO :: 1700 documents